# Idiot
https://habrahabr.ru/post/342738/

In [1]:
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf
from dataset import Dataset

In [2]:
dataset = Dataset()
dataset.load('data/anna.txt')
model_path = 'models/002/model.ckpt'

In [9]:
def build_inputs():
    inputs  = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [10]:
def build_lstm(lstm_size, num_layers, keep_prob):
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    return cell

In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' Строим softmax слой и возвращаем результат его работы.
    
        Аргументы
        ---------
        
        x: Входящий от LSTM тензор
        in_size: Размер входящего тензора, (кол-во LSTM юнитов скрытого слоя)
        out_size: Размер softmax слоя (объем словаря)
    
    '''

    # вытягиваем и решэйпим тензор, выполняя преобразование 3D -> 2D
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Соединяем результат LTSM слоев с softmax слоем
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Считаем logit-функцию
    logits = tf.matmul(x, softmax_w) + softmax_b
    # Используем функцию softmax для получения предсказания
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Считаем функцию потери на основании значений logit-функции и целевых значений.
    
        Аргументы
        ---------
        logits: значение logit-функции
        targets: целевые значения, с которыми сравниваем предсказания
        lstm_size: Количество юнитов в LSTM слое
        num_classes: Количество классов в целевых значениях (размер словаря)
        
    '''
    # Делаем one-hot кодирование целевых значений и решейпим по образу и подобию logits
    y_one_hot = tf.one_hot(targets, num_classes)
    # y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Считаем значение функции потери softmax cross entropy loss и возвращаем среднее значение
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot)
    loss = tf.reduce_mean(loss)
    return loss

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Строим оптимизатор для обучения, используя обрезку градиента.
    
        Arguments:
        loss: значение функции потери
        learning_rate: параметр скорости обучения
    
    '''
    
    # Оптимизатор для обучения, обрезка градиента для контроля "взрывающихся" градиентов
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, lstm_size=128, num_layers=2, learning_rate=0.001, grad_clip=5):

        tf.reset_default_graph()
        
        # Получаем input placeholder'ы
        self.inputs, self.targets, self.keep_prob = build_inputs()

        # Строим LSTM ячейку
        cell = build_lstm(lstm_size, num_layers, self.keep_prob)

        ### Прогоняем данные через RNN слои
        # Делаем one-hot кодирование входящих данных
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Прогоняем данные через RNN и собираем результаты
        self.seq_length_pl = tf.placeholder(tf.int32, [None], 'seq_length')
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, sequence_length=self.seq_length_pl, dtype=tf.float32)
        self.final_state = state

        # Получаем предсказания (softmax) и результат logit-функции
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        

        # Считаем потери и оптимизируем (с обрезкой градиента)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [15]:
batch_size = 100        # Размер пакета
num_steps = 100         # Шагов в пакете
lstm_size = 512         # Количество LSTM юнитов в скрытом слое
num_layers = 2          # Количество LSTM слоев
learning_rate = 0.001   # Скорость обучения
keep_prob = 0.5         # Dropout keep probability

In [ ]:
%%time
epochs = 50
# Сохраняться каждый N итераций
save_every_n = 100
vocab = dataset.vocab

model = CharRNN(len(vocab), 
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver()
sess  = tf.Session()
try:
    sess.run(tf.global_variables_initializer())

    try:
        saver.restore(sess, model_path)
    except:
        print("no data at", model_path)
    counter = 0
    for e in range(epochs):
        # Обучаем сеть
        loss = 0
        for x, y in dataset.get_batches(batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.seq_length_pl: [50, ]*batch_size,
                    model.keep_prob: keep_prob}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)

            end = time.time()

            if (counter % save_every_n == 0):
                saver.save(sess, model_path)
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
except KeyboardInterrupt:
    pass

INFO:tensorflow:Restoring parameters from models/002/model.ckpt
Epoch: 1/50...  Training Step: 100...  Training loss: 3.0510...  0.3174 sec/batch
Epoch: 2/50...  Training Step: 200...  Training loss: 2.9471...  0.3797 sec/batch
Epoch: 2/50...  Training Step: 300...  Training loss: 2.9032...  0.3618 sec/batch
Epoch: 3/50...  Training Step: 400...  Training loss: 2.8489...  0.3292 sec/batch
Epoch: 3/50...  Training Step: 500...  Training loss: 2.8093...  0.2285 sec/batch
Epoch: 4/50...  Training Step: 600...  Training loss: 2.7406...  0.2734 sec/batch
Epoch: 5/50...  Training Step: 700...  Training loss: 2.7020...  0.2926 sec/batch
Epoch: 5/50...  Training Step: 800...  Training loss: 2.6888...  0.4158 sec/batch
Epoch: 6/50...  Training Step: 900...  Training loss: 2.6460...  0.3675 sec/batch
Epoch: 6/50...  Training Step: 1000...  Training loss: 2.6411...  0.3125 sec/batch
Epoch: 7/50...  Training Step: 1100...  Training loss: 2.6187...  0.2596 sec/batch
Epoch: 8/50...  Training Step: 1

In [ ]:
# TDO: Initial state! and state for learing and generation

In [17]:
def pick_top_n(preds, vocab_size, top_n):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [18]:
def sample(model, n_samples, lstm_size, vocab_size, top_n, prime="Глаза его были большие"):
    samples = [c for c in prime]
    for c in prime:
        x = np.zeros((1, 1))
        x[0,0] = dataset.vocab_to_int[c]
        feed = {model.inputs: x,
                model.seq_length_pl: [1, ]*1,
                model.keep_prob: 1.}
        preds, new_state = sess.run([model.prediction, model.final_state], feed_dict=feed)

    c = pick_top_n(preds, len(dataset.vocab), top_n)
    samples.append(dataset.int_to_vocab[c])

    for i in range(n_samples):
        x[0,0] = c
        feed = {model.inputs: x,
                model.seq_length_pl: [1,]*1,
                model.keep_prob: 1.}
        preds, new_state = sess.run([model.prediction, model.final_state], feed_dict=feed)

        c = pick_top_n(preds, len(dataset.vocab), top_n)
        samples.append(dataset.int_to_vocab[c])
        
    return ''.join(samples)

In [19]:
# samp = sample(1000, lstm_size, len(vocab), top_n=5, prime=)
primes = ['Доколе математики будут уверены в непогрешимости своего мозга, и доказательств, оттуда выходящих?']
for prime in primes:
    print('-'*50)
    samp = sample(model, 500, lstm_size, len(dataset.vocab), top_n=5, prime=prime)
    samp = samp.replace('\n', '\n\n')
    print(samp)

--------------------------------------------------
Доколе математики будут уверены в непогрешимости своего мозга, и доказательств, оттуда выходящих?

 сто о пи оне се нотатескоса пето сеноти нулетонусла остестоти по пали нитове о вонитете оноталаско састрена пестоте во пра насетина оно слосатеноно нал по сто слоно пото не но во ни о нескотри скиното востени валалититов пра о но овито ни оселало се васлино пи ве ве нину повостоноста скателатостостото нуленалоно пра палиласлалета ноте пеле о проно скатовове ски ве сти се восла стресенелони сла, ск нала, пона, селалини нисалетотисе не ва, но понели витра оста прустова прало оскосло на, насло пал
